In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# #이후 런타임 다시 시작

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directory
fc-cache: succeeded


In [ ]:
import warnings   
warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('font', family='NanumBarunGothic') 
plt.rcParams["font.size"] = 12
plt.rcParams['xtick.labelsize'] = 12.
plt.rcParams['ytick.labelsize'] = 12.
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
import numpy as np
import pandas as pd
import re


from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import  accuracy_score, precision_score, recall_score, f1_score

In [ ]:
cd /content/drive/MyDrive/UBION_final_project/data

/content/drive/MyDrive/UBION_final_project/data


# show_result, store_result

In [ ]:
def show_result(y_test, prediction):
  global condition
  CM = confusion_matrix(y_test, prediction) #Confusion_matrix
  A = round( accuracy_score(y_test,prediction),3)  # 정확도
  P = round( precision_score(y_test, prediction),3)  #정밀도
  R = round( recall_score(y_test, prediction), 3) #재현율
  F = round( f1_score(y_test, prediction), 3) #f1-score
  
  condition.append(CM)
  condition.append(A)
  condition.append(P)
  condition.append(R)
  condition.append(F)

In [ ]:
# cutoff_result = pd.DataFrame(columns = ['데이터프레임', 'train_test_split(Y/R)', 't-test(Y/N)', '변수선택법', 'VIF(Y/N)','sampling 방식', '적용모델','선택된 변수','혼동행렬', 'Accuracy', 'Precision', 'Recall', 'F1-score','hyperparameter 변경'])
# cutoff_result.to_csv('cutoff_result.csv', index=False)

In [ ]:
def store_result(condition):
    cutoff_result = pd.read_csv('cutoff_result.csv')

    condition= pd.DataFrame({len(cutoff_result): condition}).transpose()
    condition.columns = ['데이터프레임', 'train_test_split(Y/R)', 't-test(Y/N)', '변수선택법', 'VIF(Y/N)','sampling 방식', '적용모델','선택된 변수','혼동행렬', 'Accuracy', 'Precision', 'Recall', 'F1-score','hyperparameter 변경']
    

    cutoff_result = pd.concat([cutoff_result, condition], axis=0, ignore_index=True)

    cutoff_result.to_csv('cutoff_result.csv', index=False)

# Excute_model

In [ ]:
def change_cutoff(result_df):
    value_00 = result_df[(result_df['Actual']==0) & (result_df['prediction']== 0)]['부도_prob'].max()
    value_01 = result_df[(result_df['Actual']==0) & (result_df['prediction']== 1)]['부도_prob'].min()
    value_10 = result_df[(result_df['Actual']==1) & (result_df['prediction']== 0)]['부도_prob'].max()
    value_11 = result_df[(result_df['Actual']==1) & (result_df['prediction']== 1)]['부도_prob'].min()

    if value_10 < value_00:
        cutoff = value_11
        result_df.loc[(result_df['Actual']==0) & (result_df['prediction']== 1) & (result_df['부도_prob']<cutoff) , 'prediction'] = 0
    
    if value_11 > value_01:
        cutoff= value_00
        result_df.loc[(result_df['Actual']==1) & (result_df['prediction']== 0) & (result_df['부도_prob']>cutoff) , 'prediction'] = 1

    return result_df

In [ ]:
def Excute_lg_change_cutoff (X_train, y_train, X_test, y_test):
    lr = LogisticRegression(random_state=0)
    lr.fit(X_train, y_train) 
    prediction = lr.predict(X_test)
    prediction_prob =lr.predict_proba(X_test)

    y_test = pd.Series(y_test.values)
    y_test = y_test.to_frame()

    result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
    result_df['prediction'] = prediction
    result_df['Actual'] = y_test

    return_df = change_cutoff(result_df)
    show_result(result_df['Actual'], result_df['prediction'])

In [ ]:
def Excute_sgd_change_cutoff(X_train, y_train, X_test, y_test):
    sgd = SGDClassifier(random_state=0)
    sgd.fit(X_train, y_train)
    prediction = sgd.predict(X_test)
    prediction_prob =sgd.predict_proba(X_test)

    y_test = pd.Series(y_test.values)
    y_test = y_test.to_frame()

    result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
    result_df['prediction'] = prediction
    result_df['Actual'] = y_test


    return_df = change_cutoff(result_df)
    show_result(result_df['Actual'], result_df['prediction'])

In [ ]:
def Excute_svm_change_cutoff(X_train, y_train, X_test, y_test):
    svc = SVC(random_state=0)
    svc.fit(X_train, y_train)
    prediction = svc.predict(X_test)
    prediction_prob =svc.predict_proba(X_test)

    y_test = pd.Series(y_test.values)
    y_test = y_test.to_frame()

    result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
    result_df['prediction'] = prediction
    result_df['Actual'] = y_test


    return_df = change_cutoff(result_df)
    show_result(result_df['Actual'], result_df['prediction'])

In [ ]:
def Excute_pr_change_cutoff(X_train, y_train, X_test, y_test):
    dt2 = DecisionTreeClassifier(max_depth=2)
    dt2.fit(X_train, y_train)
    prediction = dt2.predict(X_test)
    prediction_prob =dt2.predict_proba(X_test)

    y_test = pd.Series(y_test.values)
    y_test = y_test.to_frame()

    result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
    result_df['prediction'] = prediction
    result_df['Actual'] = y_test


    return_df = change_cutoff(result_df)
    show_result(result_df['Actual'], result_df['prediction'])

In [ ]:
def Excute_dt_change_cutoff(X_train, y_train, X_test, y_test):
    dt = DecisionTreeClassifier(random_state=0)
    dt.fit(X_train, y_train)
    prediction = dt.predict(X_test)
    prediction_prob =dt.predict_proba(X_test)

    y_test = pd.Series(y_test.values)
    y_test = y_test.to_frame()

    result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
    result_df['prediction'] = prediction
    result_df['Actual'] = y_test


    return_df = change_cutoff(result_df)
    show_result(result_df['Actual'], result_df['prediction'])

In [ ]:
def Excute_knn_change_cutoff(X_train, y_train, X_test, y_test):
    knn = KNeighborsClassifier()
    knn.fit(X_train, y_train)
    prediction = knn.predict(X_test)
    prediction_prob =knn.predict_proba(X_test)

    y_test = pd.Series(y_test.values)
    y_test = y_test.to_frame()

    result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
    result_df['prediction'] = prediction
    result_df['Actual'] = y_test


    return_df = change_cutoff(result_df)
    show_result(result_df['Actual'], result_df['prediction'])

In [ ]:
def Excute_rf_change_cutoff(X_train, y_train, X_test, y_test):
    rfc = RandomForestClassifier(random_state=0)
    rfc.fit(X_train, y_train)
    prediction = rfc.predict(X_test)
    prediction_prob =rfc.predict_proba(X_test)

    y_test = pd.Series(y_test.values)
    y_test = y_test.to_frame()

    result_df = pd.DataFrame(prediction_prob, columns=['정상_prob','부도_prob'])
    result_df['prediction'] = prediction
    result_df['Actual'] = y_test


    return_df = change_cutoff(result_df)
    show_result(result_df['Actual'], result_df['prediction'])

# sampling

In [ ]:
def RUSE(X_train, y_train):
  from imblearn.under_sampling import RandomUnderSampler
  rus = RandomUnderSampler()
  
  X_undersampled, y_undersampled = rus.fit_resample(X_train, y_train)
  
  return X_undersampled, y_undersampled

In [ ]:
def ROSE(X_train, y_train):
  from imblearn.over_sampling import RandomOverSampler
  ros = RandomOverSampler()

  X_oversampled, y_oversampled = ros.fit_resample(X_train, y_train)
  
  return X_oversampled, y_oversampled

In [ ]:
def SMOTE(X_train, y_train):
  from imblearn.over_sampling import SMOTE
  smote = SMOTE()
  X_smoted, y_smoted = smote.fit_resample(X_train,y_train)
  
  return X_smoted, y_smoted

# train/test/split

In [ ]:
def split_by_Y(df):
    after_2018  = df.loc[df['회계년도'] >= 2018]['차기부도여부'].value_counts()[1]
    before_2018 = df.loc[df['회계년도'] <  2018]['차기부도여부'].value_counts()[1]
    SIZE = after_2018/len(df)

    test = df[df['회계년도'] >= 2018]
    train = df[df['회계년도'] < 2018]
    
    test.drop(columns = ['회계년도'], inplace=True)
    train.drop(columns= ['회계년도'], inplace=True)

    X_train = train.drop(['차기부도여부'], axis= 1)
    y_train = train['차기부도여부']

    X_test = test.drop(['차기부도여부'], axis=1)
    y_test = test['차기부도여부']

    return X_train, y_train, X_test, y_test

In [ ]:
def split_by_R(df):
    from sklearn.model_selection import train_test_split
    
    data = df.drop(['차기부도여부'], axis= 1)
    target = df['차기부도여부']

    # train_test_split
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.178, shuffle=True, stratify=target, random_state=34)
    X_test.drop(columns = ['회계년도'], inplace=True)
    X_train.drop(columns= ['회계년도'], inplace=True)

    return X_train, y_train, X_test, y_test

# main

In [ ]:
def RUN_ALL(filename, seleted_variables, train_test_split_method , sampling_method, model_name):
    df = pd.read_csv(filename+'.csv')
    global condition

    df = df[seleted_variables]

    if train_test_split_method == 'Y' :
        X_train, y_train, X_test, y_test = split_by_Y(df)
        print(X_train.head(2))
    elif train_test_split_method == 'R' :
        X_train, y_train, X_test, y_test = split_by_R(df)

    if sampling_method == 'RUSE' :
        sampled_X_train, sampled_y_train = RUSE( X_train, y_train )
    elif sampling_method =='ROSE' :
        sampled_X_train, sampled_y_train = ROSE( X_train, y_train )
    elif sampling_method =='SMOTE' :
        sampled_X_train, sampled_y_train = SMOTE( X_train, y_train)
    elif sampling_method =='None':
        sampled_X_train, sampled_y_train = X_train, y_train


    if model_name =='lg':
        Excute_lg_change_cutoff(sampled_X_train, sampled_y_train, X_test, y_test)
    elif model_name =='sgd':
        Excute_sgd_change_cutoff(sampled_X_train, sampled_y_train, X_test, y_test)
    elif model_name =='knn':
        Excute_knn_change_cutoff(sampled_X_train, sampled_y_train, X_test, y_test)
    elif model_name =='svm':
        Excute_svm_change_cutoff(sampled_X_train, sampled_y_train, X_test, y_test)
    elif model_name =='dt':
        Excute_dt_change_cutoff(sampled_X_train, sampled_y_train, X_test, y_test)
    elif model_name =='pr':
        Excute_pr_change_cutoff(sampled_X_train, sampled_y_train, X_test, y_test)
    elif model_name =='rf':
        Excute_rf_change_cutoff(sampled_X_train, sampled_y_train, X_test, y_test)

In [ ]:
target = pd.read_csv('target.csv')
target= target[target['적용모델']!='sgd']
import pprint

In [ ]:
for idx, col in target.iterrows():
    condition = col[:8].to_list()

    seleted_variables = condition[7]
    seleted_variables = re.sub("\[|\]|\'|\'|", "",seleted_variables)
    seleted_variables = list(seleted_variables.split(', '))
    seleted_variables.append('차기부도여부')
    seleted_variables.append('회계년도')

    print("***", condition[:7])
    RUN_ALL(condition[0], seleted_variables, condition[1] , condition[5], condition[6])
    condition.append('cutoff')
    store_result(condition)

*** ['small_business_df', 'R', 't_Y', 'Lasso', 'v_N', 'SMOTE', 'lg']
*** ['small_business_df', 'R', 't_Y', 'forward', 'v_N', 'ROSE', 'lg']


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


*** ['small_business_df', 'R', 't_Y', 'Lasso', 'v_Y', 'RUSE', 'rf']
*** ['small_business_df', 'R', 't_Y', 'stepwise', 'v_N', 'RUSE', 'rf']
*** ['diff_df', 'R', 't_Y', 'forward', 'v_N', 'RUSE', 'rf']
*** ['small_business_df', 'R', 't_Y', 'forward', 'v_N', 'RUSE', 'lg']


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


*** ['small_business_df', 'R', 't_Y', 'forward', 'v_N', 'SMOTE', 'lg']


...

*** ['small_business_df', 'R', 't_Y', 'stepwise', 'v_N', 'SMOTE', 'lg']


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


*** ['small_business_df', 'R', 't_Y', 'Lasso', 'v_N', 'ROSE', 'lg']
*** ['small_business_df', 'R', 't_Y', 'stepwise', 'v_N', 'ROSE', 'lg']


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


*** ['small_business_df', 'R', 't_Y', 'Lasso', 'v_N', 'RUSE', 'lg']
*** ['small_business_df', 'R', 't_Y', 'None', 'v_N', 'SMOTE', 'lg']
*** ['small_business_df', 'R', 't_Y', 'None', 'v_Y', 'RUSE', 'rf']
*** ['small_business_df', 'R', 't_Y', 'None', 'v_N', 'RUSE', 'rf']
